In [273]:
import os
import math
import pickle
import pandas
import plotly
import numpy as np
import matplotlib.pyplot as plt


# sklearn basic
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# sklearn model
from sklearn import tree
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import BayesianRidge
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import RadiusNeighborsRegressor


#XGboost
import xgboost as xgb


# ensemble learning
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor




pandas.set_option('display.max_columns', None)

**Read CSV**

In [2]:
df = pandas.read_csv("train.csv", header = 0)
df = df.drop(columns = ["Street", "Utilities", "PoolQC", "GarageYrBlt"])
# df = df.drop(columns = ["Street", "Utilities", "PoolQC", "GarageYrBlt", "MoSold", "YrSold", "YearBuilt", "YearRemodAdd"])

**Result Table**

In [256]:
Result_list = []

**Convert to one-hot**
- MSSubClass, MSZoning, Street, LotConfig, LandSlope, Neighborhood, Condition1, Condition2, BldgType, HouseStyle, RoofStyle, RoofMatl, Exterior1st, Exterior2nd, MasVnrType, Foundation, Heating, Electrical, Functional, GarageType, PavedDrive, Fence, MiscFeature, SaleType, SaleConditionc, Alley

In [3]:
cate_list =["Alley","MSSubClass", "MSZoning", "LotConfig", "LandSlope", "Neighborhood", "Condition1", "Condition2", "BldgType", "HouseStyle", "RoofStyle", "RoofMatl", "Exterior1st", "Exterior2nd", "MasVnrType", "Foundation", "Heating", "Electrical", "Functional", "GarageType", "PavedDrive", "Fence", "MiscFeature", "SaleType", "SaleCondition"]

In [4]:
for c in cate_list:
    lis = []
    for i in df[c].unique():
        if(not isinstance(i, float)):
#             if(not math.isnan(i)):
            lis.append((i))
        
    Type = pandas.DataFrame(columns = sorted(lis, reverse = False)) # create the new dataframe base the all the type key found
    key = list(Type.keys()) # loop over all the types
    count = 0

    for i in df[c].tolist():
        value_list = np.zeros(len(key)).reshape(1,len(key)) # default all 0
        value_list = pandas.DataFrame(value_list, columns = key)
        Type = Type.append(value_list) # append the new row into dataframe
        Type.iloc[count][(i)] = 1 # the select type feature will be 1
        count += 1 # index the current row
        
    for i in Type.keys():
        df[str(c)+str(i)] = Type[i].tolist()
df = df.drop(columns = cate_list)

**Convert to Boolean**

In [5]:
CA_list = df["CentralAir"].tolist()
CA_Convert_list = []
for i in CA_list:
    if(i == "N"):
        CA_Convert_list.append(0)
    else:
        CA_Convert_list.append(1)
df["CentralAir_New"] = CA_Convert_list
df = df.drop(columns = ["CentralAir"])

**Convert to Rank**

- LotShape, LandContour, ExterQual, ExterCond, BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, HeatingQC, KitchenQual, FireplaceQu, GarageFinish, GarageQual, GarageCond, PoolQC

- OverallQual, OverallCond

In [6]:
LotShape_Convert_list = []
for i in df["LotShape"].tolist():
    if(i == "Reg"):
        LotShape_Convert_list.append(3)
    elif(i == "IR1"):
        LotShape_Convert_list.append(2)
    elif(i == "IR2"):
        LotShape_Convert_list.append(1)
    elif(i == "IR3"):
        LotShape_Convert_list.append(0)
df["LotShape_New"] = LotShape_Convert_list
df = df.drop(columns = ["LotShape"])

In [7]:
LandContour_Convert_list = []
for i in df["LandContour"].tolist():
    if(i == "Lvl"):
        LandContour_Convert_list.append(3)
    elif(i == "Bnk"):
        LandContour_Convert_list.append(2)
    elif(i == "HLS"):
        LandContour_Convert_list.append(1)
    elif(i == "Low"):
        LandContour_Convert_list.append(0)
df["LandContour_New"] = LandContour_Convert_list
df = df.drop(columns = ["LandContour"])

In [8]:
for k in ["ExterCond", "ExterQual", "HeatingQC", "KitchenQual"]:
    ExterQual_Convert_list = []
    for i in df[k].tolist():
        if(i == "Ex"):
            ExterQual_Convert_list.append(4)
        elif(i == "Gd"):
            ExterQual_Convert_list.append(3)
        elif(i == "TA"):
            ExterQual_Convert_list.append(2)
        elif(i == "Fa"):
            ExterQual_Convert_list.append(1)
        elif(i == "Po"):
            ExterQual_Convert_list.append(0)
    df[str(k)+"_New"] = ExterQual_Convert_list
    df = df.drop(columns = [k])

In [9]:
for k in ["BsmtExposure"]:
    ExterQual_Convert_list = []
    for i in df[k].tolist():
        if(not isinstance(i, float)):
            if(i == "Gd"):
                ExterQual_Convert_list.append(4)
            elif(i == "Av"):
                ExterQual_Convert_list.append(3)
            elif(i == "Mn"):
                ExterQual_Convert_list.append(2)
            elif(i == "No"):
                ExterQual_Convert_list.append(1)
        else:
            ExterQual_Convert_list.append(0)


    df[str(k)+"_New"] = ExterQual_Convert_list
    df = df.drop(columns = [k])

In [10]:
for k in ["BsmtCond", "BsmtQual", "FireplaceQu", "GarageQual", "GarageCond"]:
    BsmtQual_Convert_list = []
    for i in df[k].tolist():
        if(not isinstance(i, float)):
            if(i == "Ex"):
                BsmtQual_Convert_list.append(5)
            elif(i == "Gd"):
                BsmtQual_Convert_list.append(4)
            elif(i == "TA"):
                BsmtQual_Convert_list.append(3)
            elif(i == "Fa"):
                BsmtQual_Convert_list.append(2)
            elif(i == "Po"):
                BsmtQual_Convert_list.append(1)
            elif(i == "NA"):
                BsmtQual_Convert_list.append(0)
        else:
            BsmtQual_Convert_list.append(0)


    df[str(k)+"_New"] = BsmtQual_Convert_list
    df = df.drop(columns = [k])

In [11]:
for k in ["BsmtFinType1", "BsmtFinType2"]:
    BsmtQual_Convert_list = []
    for i in df[k].tolist():
        if(not isinstance(i, float)):
            if(i == "GLQ"):
                BsmtQual_Convert_list.append(6)
            elif(i == "ALQ"):
                BsmtQual_Convert_list.append(5)
            elif(i == "Rec"):
                BsmtQual_Convert_list.append(4)
            elif(i == "BLQ"):
                BsmtQual_Convert_list.append(3)
            elif(i == "LwQ"):
                BsmtQual_Convert_list.append(2)
            elif(i == "Unf"):
                BsmtQual_Convert_list.append(1)
            elif(i == "NA"):
                BsmtQual_Convert_list.append(0)
        else:
            BsmtQual_Convert_list.append(0)


    df[str(k)+"_New"] = BsmtQual_Convert_list
    df = df.drop(columns = [k])


In [12]:
for k in ["GarageFinish"]:
    BsmtQual_Convert_list = []
    for i in df[k].tolist():
        if(not isinstance(i, float)):
            if(i == "Fin"):
                BsmtQual_Convert_list.append(3)
            elif(i == "RFn"):
                BsmtQual_Convert_list.append(2)
            elif(i == "Unf"):
                BsmtQual_Convert_list.append(1)
            elif(i == "NA"):
                BsmtQual_Convert_list.append(0)
        else:
            BsmtQual_Convert_list.append(0)
df[str(k)+"_New"] = BsmtQual_Convert_list
df = df.drop(columns = [k])

In [13]:
# replace nan to 0
df["LotFrontage"] = df["LotFrontage"].fillna(0).tolist()
df["MasVnrArea"] = df["MasVnrArea"].fillna(0).tolist()
df = df.drop(columns = ["Id"])


In [14]:
scale_df = df.drop(columns = ["SalePrice"])
keys = list(scale_df.keys())

scaler = MinMaxScaler()
MinMaxScaler(copy=True, feature_range=(0, 1))
scaler.fit(scale_df)
scale_df = scaler.transform(scale_df)
scale_df = pandas.DataFrame(scale_df, columns = keys)
scale_df["SalePrice"] = df["SalePrice"].tolist()


In [15]:
df = scale_df

**PCA**

In [ ]:
candidate_components = range(10, 240, 30)
explained_ratios = []
for c in candidate_components:
    pca = PCA(n_components=c)
    X_pca = pca.fit_transform(df.drop(columns = ["SalePrice"]))
    explained_ratios.append(np.sum(pca.explained_variance_ratio_))

plt.figure(figsize=(10, 6), dpi=144)
plt.grid()
plt.plot(candidate_components, explained_ratios)
plt.xlabel('Number of PCA Components')
plt.ylabel('Explained Variance Ratio')
plt.title('Explained variance ratio for PCA')
plt.yticks(np.arange(0.5, 1.05, .05))
plt.xticks(np.arange(0, 300, 20))

In [ ]:
Y = df["SalePrice"].tolist()
X = df.drop(columns = ["SalePrice"]).values.tolist()
pca = sklearn.decomposition.PCA(n_components=100)
pca.fit(X)
X = pca.transform(X)


In [ ]:
X.shape

**Dataset**

In [21]:
# sklearn split
Train_X, Test_X, Train_Y, Test_Y = train_test_split(
    df.drop(columns = "SalePrice").values.tolist(), 
    df["SalePrice"].tolist(),
    test_size=0.07, 
    random_state=42)

# stacking dataset or vaild for boosting
Train_Stacking_X = Train_X[:150]
Train_Stacking_Y = Train_Y[:150]

Train_X = Train_X[150:]
Train_Y = Train_Y[150:]

In [ ]:
# pandas shuffle
dataset = shuffle(df)
Test = dataset[:100]
Train = dataset[100:]

Train_Stacking = Train[:300]
Train = Train[300:]

Test_X = Test.drop(columns = "SalePrice").values.tolist()
Test_Y = Test["SalePrice"].tolist()

Train_X = Train.drop(columns = "SalePrice").values.tolist()
Train_Y = Train["SalePrice"].tolist()

Train_Stacking_X = Train_Stacking.drop(columns = "SalePrice").values.tolist()
Train_Stacking_Y = Train_Stacking["SalePrice"].tolist()

In [ ]:
# After PCA
Test = X[:100]
Train = X[100:]

Test_X = Test
Test_Y = Y[:100]

Train_X = Train
Train_Y = Y[100:]

## Machine Learning Model

In [16]:
rng = np.random.RandomState(1)

**Adam Boost**

In [17]:
# Adam boost for the model and return the best selection via given datasets
def Ada_Model(model, Train_X, Train_Y, Vail_X, Vail_Y):
    highest = 0
    coeff = 0
    rng = np.random.RandomState(1)
    for i in range(1, 20):
        AdaBoost_model = AdaBoostRegressor(base_estimator=model, n_estimators=i, random_state=rng)
        AdaBoost_model.fit(Train_X, Train_Y)
        curr_score = AdaBoost_model.score(Vail_X, Vail_Y)
        if(highest < curr_score):
            highest = curr_score
            coeff = i
    return coeff, highest


**Bagging**

In [18]:
def Bagging(model, Train_X, Train_Y, Vail_X, Vail_Y):  
    highest = 0
    coeff = 0
    rng = np.random.RandomState(1)
    for i in range(1, 20):
        Bagging_Model = BaggingRegressor(base_estimator= model, n_estimators = i, bootstrap=True, warm_start=False) #
        Bagging_Model.fit(Train_X, Train_Y)
        curr_score = Bagging_Model.score(Vail_X, Vail_Y)
        if(highest < curr_score):
            highest = curr_score
            coeff = i
    return coeff, highest

# sklearn.ensemble.BaggingRegressor(base_estimator=None, 
# n_estimators=10, 
# max_samples=1.0, 
# max_features=1.0, 
# bootstrap=True, 
# bootstrap_features=False, 
# oob_score=False, 
# warm_start=False, 
# n_jobs=None, 
# random_state=None, verbose=0)

**Gradient Boosting**

In [19]:
def GBDT(model, Train_X, Train_Y, Vail_X, Vail_Y):
    coeff = 0
    highest = 0
    _range = np.arange(0.01, 1, 0.05)
    for i in (_range):
        GBDT = GradientBoostingRegressor(init = model, learning_rate = i)
        GBDT.fit(Train_X, Train_Y)
        curr_score = GBDT.score(Train_Stacking_X, Train_Stacking_Y)
        if(highest < curr_score):
            highest = curr_score
            coeff = i
    return coeff, highest

**Neural Network**

In [22]:
NN = MLPRegressor(solver='adam',  # weight optimization
                    activation='relu',  # activation function
                    alpha=1e-2, # Regularizer value
                    learning_rate_init = 0.1,
                    hidden_layer_sizes=(30,20,5), # layer and node (node,node) --> numbers of layers

                  random_state=1, #random state
                    max_iter = 1000, # max iteration
                    early_stopping = True) # early stop

NN.fit(Train_X, Train_Y) # fit the neural network


MLPRegressor(activation='relu', alpha=0.01, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=True, epsilon=1e-08,
             hidden_layer_sizes=(30, 20, 5), learning_rate='constant',
             learning_rate_init=0.1, max_iter=1000, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=1, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [23]:
NN.score(Train_X, Train_Y)

0.8765227063911397

In [24]:
NN.score(Test_X, Test_Y)

0.8621464009981835

In [26]:
coeff, highest = Bagging(NN, Train_X, Train_Y, Train_Stacking_X, Train_Stacking_Y)
coeff, highest

(1, 0.9046479031201354)

In [27]:
Bagging_NN = BaggingRegressor(base_estimator= NN, n_estimators = coeff, bootstrap=True, warm_start=False) #
Bagging_NN.fit(Train_X, Train_Y)
Bagging_NN.score(Train_X, Train_Y)

0.8048061031018695

In [28]:
Bagging_NN.score(Test_X, Test_Y)

0.8805444810240521

In [112]:
# Boosting Net
coeff, highest = Ada_Model(Bagging_NN, Train_X, Train_Y, Train_Stacking_X, Train_Stacking_Y)
coeff, highest

(18, 0.9027425306378715)

In [113]:
# for i in range(0,3):
AdaBoost_NN = AdaBoostRegressor(base_estimator=Bagging_NN, n_estimators=coeff, random_state=rng)
AdaBoost_NN.fit(Train_X, Train_Y)
AdaBoost_NN.score(Train_X, Train_Y)

0.9764107005334632

In [114]:
AdaBoost_NN.score(Test_X,Test_Y)

0.8576248515610712

In [115]:
coeff, highest = GBDT(Bagging_NN, Train_X, Train_Y, Train_Stacking_X, Train_Stacking_Y)
coeff, highest

(0.51, 0.9344089223649689)

In [116]:
GBDT_NN = GradientBoostingRegressor(init = Bagging_NN, learning_rate = coeff)
GBDT_NN.fit(Train_X, Train_Y)
GBDT_NN.score(Train_X, Train_Y)

0.9934266867223877

In [117]:
GBDT_NN.score(Test_X, Test_Y)

0.9171422441659779

In [257]:
Result_list.append(["NN", 
                    NN.score(Test_X, Test_Y), 
                    Bagging_NN.score(Test_X, Test_Y),
                    AdaBoost_NN.score(Test_X, Test_Y), 
                    GBDT_NN.score(Test_X, Test_Y)])

In [276]:
# save the model to disk
lis = [NN, Bagging_NN, AdaBoost_NN, GBDT_NN]
lis_tag = ["NN", "Bagging_NN", "AdaBoost_NN", "GBDT_NN"]
for i in range(0,len(lis)):
    filename = os.getcwd()+'/NN Model/'+str(lis_tag[i])+'.sav'
    pickle.dump(lis[i], open(filename, 'wb'))




**Elastic Net + Adam Boost**

In [125]:
Net = ElasticNet(random_state=0, warm_start = True)
Net.fit(Train_X, Train_Y)  
Net.score(Train_X,Train_Y)

0.608132163178565

In [126]:
Net.score(Test_X,Test_Y)

0.6134309769745292

In [127]:
coeff, highest = Bagging(Net, Train_X, Train_Y, Train_Stacking_X, Train_Stacking_Y)
coeff, highest

(3, 0.5625161723910879)

In [128]:
Bagging_Net = BaggingRegressor(base_estimator =  Net, n_estimators = coeff, bootstrap=True, warm_start=False)
Bagging_Net.fit(Train_X, Train_Y)
Bagging_Net.score(Test_X, Test_Y)

0.615028073521705

In [129]:
# Boosting Net
coeff, highest = Ada_Model(Bagging_Net, Train_X, Train_Y, Train_Stacking_X, Train_Stacking_Y)
coeff, highest

(4, 0.6452908847750574)

In [130]:
# for i in range(0,3):
AdaBoost_Net = AdaBoostRegressor(base_estimator=Bagging_Net, n_estimators=coeff, random_state=rng)
AdaBoost_Net.fit(Train_X, Train_Y)
AdaBoost_Net.score(Train_X, Train_Y)

0.6664128936487873

In [131]:
AdaBoost_Net.score(Test_X,Test_Y)

0.6761796347352977

In [132]:
coeff, highest = GBDT(Bagging_Net, Train_X, Train_Y, Train_Stacking_X, Train_Stacking_Y)
coeff, highest

(0.21000000000000002, 0.9426762361825926)

In [133]:
GBDT_Net = GradientBoostingRegressor(init = Bagging_Net, learning_rate = coeff)
GBDT_Net.fit(Train_X, Train_Y)
GBDT_Net.score(Train_X, Train_Y)

0.9825080767462263

In [134]:
GBDT_Net.score(Test_X, Test_Y)

0.8948147581521146

In [258]:
Result_list.append(["Net", 
                    Net.score(Test_X, Test_Y), 
                    Bagging_Net.score(Test_X, Test_Y), 
                    AdaBoost_Net.score(Test_X, Test_Y), 
                    GBDT_Net.score(Test_X, Test_Y)])

In [281]:
# save the model to disk
lis = [Net, Bagging_Net, AdaBoost_Net, GBDT_Net]
lis_tag = ["Net", "Bagging_Net", "AdaBoost_Net", "GBDT_Net"]
for i in range(0,len(lis)):
    filename = os.getcwd()+'/Net Model/'+str(lis_tag[i])+'.sav'
    pickle.dump(lis[i], open(filename, 'wb'))





**Ridge Regression**

In [138]:
highest = 0
coeff = 0
for i in np.arange(0, 10, 0.1): # find the best regularizer value
    ridge = Ridge(alpha=i)
    ridge.fit(Train_X, Train_Y)
#     ridge.score(Train_X, Train_Y)
    curr_score = ridge.score(Train_Stacking_X, Train_Stacking_Y)
    if(highest < curr_score):
        highest = curr_score
        coeff = i

In [139]:
highest,coeff

(0.886179169216485, 0.2)

In [140]:
ridge = Ridge(alpha=coeff)
ridge.fit(Train_X, Train_Y)
ridge.score(Train_X, Train_Y)

0.9191594328003637

In [141]:
ridge.score(Test_X, Test_Y)

0.8649592609207901

In [142]:
coeff, highest = Bagging(ridge, Train_X, Train_Y, Train_Stacking_X, Train_Stacking_Y)
coeff, highest

(11, 0.8961068897314984)

In [143]:
Bagging_ridge = BaggingRegressor(base_estimator =  ridge, n_estimators = coeff, bootstrap=True, warm_start=False)
Bagging_ridge.fit(Train_X, Train_Y)
Bagging_ridge.score(Test_X, Test_Y)

0.8593121783407439

In [144]:
# Boosting Net
coeff, highest = Ada_Model(Bagging_ridge, Train_X, Train_Y, Train_Stacking_X, Train_Stacking_Y)
coeff, highest

(2, 0.8946954755455747)

In [145]:
# for i in range(0,3):
AdaBoost_ridge = AdaBoostRegressor(base_estimator=Bagging_ridge, n_estimators=coeff, random_state=rng)
AdaBoost_ridge.fit(Train_X, Train_Y)
AdaBoost_ridge.score(Train_X, Train_Y)

0.845695757517445

In [146]:
AdaBoost_ridge.score(Test_X, Test_Y)

0.8490501537122466

In [147]:
coeff, highest = GBDT(Bagging_ridge, Train_X, Train_Y, Train_Stacking_X, Train_Stacking_Y)
coeff, highest

(0.31000000000000005, 0.9404512538512528)

In [148]:
GBDT_ridge = GradientBoostingRegressor(init = ridge, learning_rate = coeff)
GBDT_ridge.fit(Train_X, Train_Y)
GBDT_ridge.score(Train_X, Train_Y)

0.9898090425116809

In [149]:
GBDT_ridge.score(Test_X, Test_Y)

0.8534430532190186

In [259]:
Result_list.append(["Ridge", 
                    ridge.score(Test_X, Test_Y), 
                    Bagging_ridge.score(Test_X, Test_Y), 
                    AdaBoost_ridge.score(Test_X, Test_Y), 
                    GBDT_ridge.score(Test_X, Test_Y)])

In [282]:
# save the model to disk
lis = [ridge, Bagging_ridge, AdaBoost_ridge, GBDT_ridge]
lis_tag = ["ridge", "Bagging_ridge", "AdaBoost_ridge", "GBDT_ridge"]
for i in range(0,len(lis)):
    filename = os.getcwd()+'/Ridge Model/'+str(lis_tag[i])+'.sav'
    pickle.dump(lis[i], open(filename, 'wb'))






**Linear Regression**

In [51]:
lr = LinearRegression()

In [52]:
lr.fit(Train_X, Train_Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [53]:
lr.score(Train_X, Train_Y)

0.9221279413928494

In [54]:
lr.score(Test_X, Test_Y)

-8.43628072406134e+19

**Decision Tree**

In [151]:
coeff = 0
highest = 0
for i in range(1,10):
    Tree = tree.DecisionTreeRegressor(max_depth = i)
    Tree.fit(Train_X, Train_Y)
    curr_score = Tree.score(Train_Stacking_X, Train_Stacking_Y)
    if(highest < curr_score):
        highest = curr_score
        coeff = i


In [152]:
highest, coeff

(0.8524756492976404, 8)

In [153]:
Tree = tree.DecisionTreeRegressor(max_depth = coeff)
Tree.fit(Train_X, Train_Y)
Tree.score(Train_X,Train_Y)

0.9644751408987077

In [154]:
Tree.score(Test_X,Test_Y)

0.7328186006279456

In [155]:
coeff, highest = Bagging(Tree, Train_X, Train_Y, Train_Stacking_X, Train_Stacking_Y)
coeff, highest

(19, 0.9030511872299127)

In [156]:
Bagging_Tree = BaggingRegressor(base_estimator =  Tree, n_estimators = coeff, bootstrap=True, warm_start=False)
Bagging_Tree.fit(Train_X, Train_Y)
Bagging_Tree.score(Train_X, Train_Y)

0.9596254745612443

In [157]:
Bagging_Tree.score(Test_X, Test_Y)

0.8829087829103388

In [158]:
coeff, highest = Ada_Model(Bagging_Tree, Train_X, Train_Y, Train_Stacking_X, Train_Stacking_Y)
coeff, highest

(19, 0.9120645437251983)

In [159]:
AdaBoost_Tree = AdaBoostRegressor(base_estimator=Bagging_Tree, n_estimators=coeff, random_state=rng)
AdaBoost_Tree.fit(Train_X, Train_Y)
AdaBoost_Tree.score(Train_X, Train_Y)

0.9822806788032007

In [160]:
AdaBoost_Tree.score(Test_X, Test_Y)

0.8897070558260104

In [161]:
coeff, highest = GBDT(Bagging_Tree, Train_X, Train_Y, Train_Stacking_X, Train_Stacking_Y)
coeff, highest

(0.060000000000000005, 0.9300214982302184)

In [162]:
GBDT_Tree = GradientBoostingRegressor(init = Bagging_Tree, learning_rate = coeff)
GBDT_Tree.fit(Train_X, Train_Y)
GBDT_Tree.score(Train_X, Train_Y)

0.9827489758559167

In [163]:
GBDT_Tree.score(Test_X, Test_Y)

0.8752864254675917

In [260]:
Result_list.append(["Tree", 
                    Tree.score(Test_X, Test_Y), 
                    Bagging_Tree.score(Test_X, Test_Y), 
                    AdaBoost_Tree.score(Test_X, Test_Y), 
                    GBDT_Tree.score(Test_X, Test_Y)])

In [283]:
# save the model to disk
lis = [Tree, Bagging_Tree, AdaBoost_Tree, GBDT_Tree]
lis_tag = ["Tree", "Bagging_Tree", "AdaBoost_Tree", "GBDT_Tree"]
for i in range(0,len(lis)):
    filename = os.getcwd()+'/Tree Model/'+str(lis_tag[i])+'.sav'
    pickle.dump(lis[i], open(filename, 'wb'))







**Bayesian Ridge**

In [165]:
BR = BayesianRidge()
BR.fit(Train_X, Train_Y)
BR.score(Train_X, Train_Y)

0.9062298411606639

In [166]:
BR.score(Test_X, Test_Y)

0.8681998400678294

In [167]:
coeff, highest = Bagging(BR, Train_X, Train_Y, Train_Stacking_X, Train_Stacking_Y)
coeff, highest

(10, 0.8953551642548773)

In [168]:
Bagging_BR = BaggingRegressor(base_estimator =  BR, n_estimators = coeff, bootstrap=True, warm_start=False)
Bagging_BR.fit(Train_X, Train_Y)
Bagging_BR.score(Train_X, Train_Y)

0.9024495582226408

In [169]:
Bagging_BR.score(Test_X, Test_Y)

0.8728112164839084

In [170]:
# Boosting Net
coeff, highest = Ada_Model(Bagging_BR, Train_X, Train_Y, Train_Stacking_X, Train_Stacking_Y)
coeff, highest

(2, 0.8958365081322586)

In [171]:
# for i in range(0,3):
AdaBoost_BR = AdaBoostRegressor(base_estimator=Bagging_BR, n_estimators=coeff, random_state=rng)
AdaBoost_BR.fit(Train_X, Train_Y)
AdaBoost_BR.score(Train_X, Train_Y)

0.8821319204180762

In [172]:
AdaBoost_BR.score(Test_X, Test_Y)

0.8168658462218485

In [173]:
coeff, highest = GBDT(Bagging_BR, Train_X, Train_Y, Train_Stacking_X, Train_Stacking_Y)
coeff, highest

(0.36000000000000004, 0.9418871539264054)

In [174]:
GBDT_BR = GradientBoostingRegressor(init = ridge, learning_rate = coeff)
GBDT_BR.fit(Train_X, Train_Y)
GBDT_BR.score(Train_X, Train_Y)

0.9913931196754711

In [175]:
GBDT_BR.score(Test_X, Test_Y)

0.8838305534346554

In [261]:
Result_list.append(["BR", 
                    BR.score(Test_X, Test_Y), 
                    Bagging_BR.score(Test_X, Test_Y), 
                    AdaBoost_BR.score(Test_X, Test_Y), 
                    GBDT_BR.score(Test_X, Test_Y)])

In [284]:
# save the model to disk
lis = [BR, Bagging_BR, AdaBoost_BR, GBDT_BR]
lis_tag = ["BR", "Bagging_BR", "AdaBoost_BR", "GBDT_BR"]
for i in range(0,len(lis)):
    filename = os.getcwd()+'/BR Model/'+str(lis_tag[i])+'.sav'
    pickle.dump(lis[i], open(filename, 'wb'))


**KNR**

In [177]:
n = 100
highest = 0
node = 0
for i in range(1,100):
    neigh = KNeighborsRegressor(n_neighbors = i, algorithm = "auto", weights = "distance") #n_neighbors=2
    neigh.fit(Train_X, Train_Y) 
#     curr_score = neigh.score(Train_X[:n], Train_Y[:n])
    curr_score = neigh.score(Train_Stacking_X, Train_Stacking_Y)
    if(highest < curr_score):
        highest = curr_score
        node = i

In [178]:
node, highest

(1, 0.7204476074441567)

In [179]:
KNR = KNeighborsRegressor(n_neighbors = node, algorithm = "auto", weights = "distance") #n_neighbors=2
KNR.fit(Train_X, Train_Y) 
KNR.score(Test_X, Test_Y)

0.7032469819887143

In [180]:
KNR.score(Test_X, Test_Y)

0.7032469819887143

In [181]:
coeff, highest = Bagging(KNR, Train_X, Train_Y, Train_Stacking_X, Train_Stacking_Y)
coeff, highest

(15, 0.7734546150189388)

In [182]:
Bagging_KNR = BaggingRegressor(base_estimator =  KNR, n_estimators = coeff, bootstrap=False, warm_start=True)
Bagging_KNR.fit(Train_X, Train_Y)
Bagging_KNR.score(Train_X, Train_Y)

1.0

In [183]:
Bagging_KNR.score(Test_X, Test_Y)

0.7032469819887143

In [184]:
coeff, highest = Ada_Model(Bagging_KNR, Train_X, Train_Y, Train_Stacking_X, Train_Stacking_Y)
coeff, highest

(7, 0.7469084608436959)

In [185]:
AdaBoost_KNR = AdaBoostRegressor(base_estimator=Bagging_KNR, n_estimators=coeff, random_state=rng)
AdaBoost_KNR.fit(Train_X, Train_Y)
AdaBoost_KNR.score(Train_X, Train_Y)

0.9920397629912991

In [186]:
AdaBoost_KNR.score(Test_X, Test_Y)

0.6999036128246037

In [187]:
coeff, highest = GBDT(Bagging_KNR, Train_X, Train_Y, Train_Stacking_X, Train_Stacking_Y)
coeff, highest

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/bagging.py:351: UserWarning:

Warm-start fitting without increasing n_estimators does not fit new trees.

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/bagging.py:351: UserWarning:

Warm-start fitting without increasing n_estimators does not fit new trees.

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/bagging.py:351: UserWarning:

Warm-start fitting without increasing n_estimators does not fit new trees.

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/bagging.py:351: UserWarning:

Warm-start fitting without increasing n_estimators does not fit new trees.

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/bagging.py:351: UserWarning:

Warm-start fitting without increasing n_estimators does not fit new trees.

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/bagging.py:351: UserWarning:

Warm-start fitting without increasing n_estimators does not fit new trees.

//anaconda3/lib/python

(0.01, 0.7204476074441567)

In [188]:
GBDT_KNR = GradientBoostingRegressor(init = Bagging_KNR, learning_rate = coeff)
GBDT_KNR.fit(Train_X, Train_Y)
GBDT_KNR.score(Train_X, Train_Y)

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/bagging.py:351: UserWarning:

Warm-start fitting without increasing n_estimators does not fit new trees.



1.0

In [189]:
GBDT_KNR.score(Test_X, Test_Y)

0.7032469819887144

In [263]:
Result_list.append(["KNR", 
                    KNR.score(Test_X, Test_Y), 
                    Bagging_KNR.score(Test_X, Test_Y),
                    AdaBoost_KNR.score(Test_X, Test_Y), 
                    GBDT_KNR.score(Test_X, Test_Y)])

In [285]:
# save the model to disk
lis = [KNR, Bagging_KNR, AdaBoost_KNR, GBDT_KNR]
lis_tag = ["KNR", "Bagging_KNR", "AdaBoost_KNR", "GBDT_KNR"]
for i in range(0,len(lis)):
    filename = os.getcwd()+'/KNR Model/'+str(lis_tag[i])+'.sav'
    pickle.dump(lis[i], open(filename, 'wb'))



**SVR**

In [98]:
svr_poly = SVR(kernel='poly', C=1e7, gamma='auto', degree=3, epsilon=.1, coef0=1)
svr_poly.fit(Train_X, Train_Y)
svr_poly.score(Train_X,Train_Y)

0.9604894461079304

In [99]:
svr_poly.score(Test_X,Test_Y)

0.9004236321539211

In [100]:
coeff, highest = Bagging(svr_poly, Train_X, Train_Y, Train_Stacking_X, Train_Stacking_Y)
coeff, highest

(6, 0.9058165076610192)

In [101]:
Bagging_svr = BaggingRegressor(base_estimator =  svr_poly, n_estimators = coeff, bootstrap=False, warm_start=True)
Bagging_svr.fit(Train_X, Train_Y)
Bagging_svr.score(Train_X, Train_Y)

0.9604894461079304

In [102]:
Bagging_svr.score(Test_X, Test_Y)

0.9004236321539211

In [103]:
coeff, highest = Ada_Model(Bagging_svr, Train_X, Train_Y, Train_Stacking_X, Train_Stacking_Y)
coeff, highest

(2, 0.9141149156152574)

In [104]:
AdaBoost_svr = AdaBoostRegressor(base_estimator=Bagging_svr, n_estimators=coeff, random_state=rng)
AdaBoost_svr.fit(Train_X, Train_Y)
AdaBoost_svr.score(Train_X, Train_Y)

0.9114257876149752

In [105]:
AdaBoost_svr.score(Test_X, Test_Y)

0.8908434877567564

In [106]:
coeff, highest = GBDT(Bagging_svr, Train_X, Train_Y, Train_Stacking_X, Train_Stacking_Y)
coeff, highest

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/bagging.py:351: UserWarning:

Warm-start fitting without increasing n_estimators does not fit new trees.

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/bagging.py:351: UserWarning:

Warm-start fitting without increasing n_estimators does not fit new trees.

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/bagging.py:351: UserWarning:

Warm-start fitting without increasing n_estimators does not fit new trees.

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/bagging.py:351: UserWarning:

Warm-start fitting without increasing n_estimators does not fit new trees.

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/bagging.py:351: UserWarning:

Warm-start fitting without increasing n_estimators does not fit new trees.

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/bagging.py:351: UserWarning:

Warm-start fitting without increasing n_estimators does not fit new trees.

//anaconda3/lib/python

(0.6100000000000001, 0.9398800562416164)

In [107]:
GBDT_svr = GradientBoostingRegressor(init = Bagging_svr, learning_rate = coeff)
GBDT_svr.fit(Train_X, Train_Y)
GBDT_svr.score(Train_X, Train_Y)

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/bagging.py:351: UserWarning:

Warm-start fitting without increasing n_estimators does not fit new trees.



0.999453621250671

In [108]:
GBDT_svr.score(Test_X, Test_Y)

0.7899992249554324

In [264]:
Result_list.append(["SVR", 
                    svr_poly.score(Test_X, Test_Y), 
                    Bagging_svr.score(Test_X, Test_Y), 
                    AdaBoost_svr.score(Test_X, Test_Y), 
                    GBDT_svr.score(Test_X, Test_Y)])

In [286]:
# save the model to disk
lis = [svr_poly, Bagging_svr, AdaBoost_svr, GBDT_svr]
lis_tag = ["svr_poly", "Bagging_svr", "AdaBoost_svr", "GBDT_svr"]
for i in range(0,len(lis)):
    filename = os.getcwd()+'/SVR Model/'+str(lis_tag[i])+'.sav'
    pickle.dump(lis[i], open(filename, 'wb'))




**XGBoost**

In [195]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                          colsample_bytree = 0.3, 
                          learning_rate = 0.1,
                          max_depth = 5, 
                          alpha = 10, 
                          n_estimators = 10)

In [250]:
xg_reg = xgb.XGBRegressor(objective = "reg:linear", reg_lambda=2)

In [251]:
xg_reg.fit(Train_X,Train_Y)

[22:55:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=70,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=2, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [252]:
xg_reg.score(Train_X, Train_Y)

0.9553974408407383

In [253]:
xg_reg.score(Test_X, Test_Y)

0.8480248024991448

## Ensemble Learning

In [ ]:
#select all, strong, and weak models to ensemble

all_models = [NN, BR, Net, AdaBoost_Net, Tree, AdaBoost_Tree, ridge, KNR, AdaBoost_KNR, svr_poly]
all_models_labels = ["NN", "BR", "Net", "AdaBoost_Net", "Tree", "AdaBoost_Tree", "Ridge", "KNR", "AdaBoost_KNR", "SVR"]

strong_models = [NN, BR, AdaBoost_Tree, ridge, svr_poly]
strong_models_labels = ["NN", "BR", "AdaBoost_Tree", "Ridge", "SVR"]

weak_models = [AdaBoost_Net, Tree, KNR]
weak_models_labels = ["AdaBoost_Net", "Tree", "KNR"]


### **Ensemble Learning - Voting**

In [ ]:
lis = []
for i in zip(all_models_labels,all_models):
    lis.append(i)

VR_all_models = VotingRegressor(estimators=lis)
VR_all_models.fit(Train_X, Train_Y)
VR_all_models.score(Train_X, Train_Y) # Training score

In [ ]:
VR_all_models.score(Test_X, Test_Y) # Testing Score

**Voting with strong models**

In [ ]:
lis = []
for i in zip(strong_models_labels,strong_models):
    lis.append(i)

VR_strong_models = VotingRegressor(estimators=lis)
VR_strong_models.fit(Train_X, Train_Y)
VR_strong_models.score(Train_X, Train_Y) # Training score

In [ ]:
VR_strong_models.score(Test_X, Test_Y) # Testing Score

**Voting with weak models**

In [ ]:
lis = []
for i in zip(weak_models_labels,weak_models):
    lis.append(i)

VR_weak_models = VotingRegressor(estimators=lis)
VR_weak_models.fit(Train_X, Train_Y)
VR_weak_models.score(Train_X, Train_Y) # Training score

In [ ]:
VR_weak_models.score(Test_X, Test_Y)

### **Ensemble Learning - Stacking**

In [ ]:
meta = MLPRegressor() # early stop

In [ ]:
stregr = StackingRegressor(regressors=all_models, 
                           meta_regressor=meta)


stregr.fit(Train_Stacking_X, Train_Stacking_Y)
stregr.score(Train_Stacking_X, Train_Stacking_Y)

In [ ]:
stregr.score(Test_X, Test_Y)

**Stacking with strong models**

In [ ]:
stack_strong = StackingRegressor(regressors=strong_models, meta_regressor=meta) # strong models
stack_strong.fit(Train_Stacking_X, Train_Stacking_Y)
stack_strong.score(Train_Stacking_X, Train_Stacking_Y)

In [ ]:
stack_strong.score(Test_X, Test_Y)

**Stacking with weak models**

In [ ]:
stregr_weak = StackingRegressor(regressors=weak_models, meta_regressor=meta) # weak models
stregr_weak.fit(Train_Stacking_X, Train_Stacking_Y)
stregr_weak.score(Train_Stacking_X, Train_Stacking_Y)

In [ ]:
stregr_weak.score(Test_X, Test_Y)

In [ ]:
Bagging_all_models.score(Test_X, Test_Y)

## Compare Result

In [266]:
Result_DF = pandas.DataFrame(Result_list, columns = ["Model Name","Raw Model", "Bagging Model", "AdaBoost Model", "GBDT Model"])
Result_DF.drop_duplicates()


,Model Name,Raw Model,Bagging Model,AdaBoost Model,GBDT Model
0,NN,0.862146,0.794817,0.857625,0.917142
1,Net,0.613431,0.608018,0.676180,0.894815
2,Ridge,0.864959,0.872336,0.849050,0.853443
3,Tree,0.732819,0.882975,0.889707,0.875286
4,BR,0.868200,0.880822,0.816866,0.883831
5,KNR,0.703247,0.703247,0.699904,0.703247
7,SVR,0.900424,0.900424,0.890843,0.789999


In [ ]:

num = 50
xt = Test_X[:num]

alpha = 0.6
plt.figure(figsize=(18,9))
plt.plot(NN.predict(xt), 'gd',alpha = alpha, label='NN')
plt.plot(BR.predict(xt), 'ko',alpha = alpha, label='BR')
plt.plot(Net.predict(xt), 'b^',alpha = alpha, label='Net')
plt.plot(neigh.predict(xt), 'b+',alpha = alpha, label='KNR')
plt.plot(Tree.predict(xt), 'cs',alpha = alpha, label='Tree')
plt.plot(ridge.predict(xt), 'yp',alpha = alpha, label='Ridge')
plt.plot(svr_poly.predict(xt), 'ys',alpha = alpha, label='SVR')

plt.plot(ereg.predict(xt), 'r*',alpha = alpha, label='VotingRegressor')
plt.plot(stregr.predict(xt), 'm*',alpha = alpha, label='StackingRegressor')


plt.plot(Test_Y[:num], 'black', label='Y_Real')


plt.tick_params(axis='x', which='both', bottom=False, top=False,
                labelbottom=False)
plt.ylabel('predicted')
plt.xlabel('training samples')
plt.legend(loc="best")
plt.title('Comparison of individual predictions with averaged')
plt.savefig('result.pdf')
plt.show()


In [ ]:
'''
Bagging - 降低variance 適合各種模型都做 ?


Adaboost - 分類問題 效果較優?
GBDT - 回歸問題 效果較優?
'''